Task 1: Use the uploaded SP500 data to carry out the PCA analysis of the PCA notebook for the years 2010 to the present. Your main challenge here is that you need to generate the corresponding SP500 price index yourself. You may use either Yahoo Finance as per the Python code or FRED as per the GARCH notebook. 

In [5]:
%pip install plotly
%pip install statsmodels
%pip install --upgrade nbformat
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import required packages
from plotly.graph_objs import Bar, Scatter, Layout, Figure
from plotly.graph_objs.layout import YAxis
from plotly.offline import iplot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
%matplotlib inline

In [7]:
# Load the csv file into dataframe
df = pd.read_csv('sp500_joined_closes.csv', index_col='Date').dropna(axis=1)
df.index = pd.to_datetime(df.index)
df.columns

Index(['MMM ', 'AOS ', 'ABT ', 'ACN ', 'ATVI ', 'ADM ', 'ADBE ', 'ADP ',
       'AAP ', 'AES ',
       ...
       'WHR ', 'WMB ', 'WTW ', 'GWW ', 'WYNN ', 'XEL ', 'YUM ', 'ZBRA ',
       'ZBH ', 'ZION '],
      dtype='object', length=438)

In [8]:
df

,MMM,AOS,ABT,ACN,ATVI,ADM,ADBE,ADP,AAP,AES,...,WHR,WMB,WTW,GWW,WYNN,XEL,YUM,ZBRA,ZBH,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,56.705299,6.113715,19.433035,32.836605,10.018278,22.393152,37.090000,27.336395,37.163261,10.011561,...,57.063725,8.963634,53.842903,76.777672,41.963718,13.504498,19.526363,28.670000,53.094780,11.041595
2010-01-05,56.350128,6.035597,19.276031,33.039528,10.036012,22.514124,37.700001,27.189604,36.942375,9.909026,...,56.945354,9.075629,53.722210,76.809219,44.515919,13.344336,19.459593,28.620001,54.775551,11.430904
2010-01-06,57.149281,6.038339,19.383074,33.390770,9.982815,22.457197,37.619999,27.125778,37.264500,9.806495,...,56.444000,9.370131,54.587097,77.101372,43.932014,13.369960,19.320469,28.400000,54.757839,12.424897
2010-01-07,57.190258,6.056155,19.543655,33.359550,9.743440,22.222376,36.889999,27.113024,37.255299,9.813819,...,57.655624,9.303761,54.426205,77.827797,44.870216,13.312304,19.314913,27.690001,56.013992,13.816483
2010-01-08,57.593262,6.145236,19.643566,33.226856,9.663650,21.944866,36.689999,27.074720,37.402538,10.092121,...,58.881119,9.378424,54.365845,78.743698,44.548733,13.318713,19.320469,27.600000,54.837471,13.592834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-06,109.930000,66.900002,102.750000,269.630005,79.629997,81.400002,347.019989,224.250015,135.130005,25.000000,...,137.240005,30.239861,239.009995,699.650024,115.400002,65.050003,129.300003,305.809998,124.620003,48.759998
2023-03-07,107.089996,66.389999,99.980003,264.200012,79.260002,79.730003,344.799988,220.033890,133.690002,24.510000,...,134.699997,29.914806,237.789993,694.669983,113.160004,63.610001,126.919998,302.160004,123.150002,46.439999
2023-03-08,107.160004,67.010002,99.400002,263.660004,79.480003,79.910004,345.839996,220.720001,132.210007,24.719999,...,136.690002,29.914806,239.080002,700.400024,113.540001,64.400002,126.779999,307.070007,121.910004,46.680000


In [9]:
# Standardize the close prices
closeNorm = (df - df.mean()) / df.std()
closeNorm

,MMM,AOS,ABT,ACN,ATVI,ADM,ADBE,ADP,AAP,AES,...,WHR,WMB,WTW,GWW,WYNN,XEL,YUM,ZBRA,ZBH,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,-1.535635,-1.446789,-1.048002,-1.102908,-1.142548,-0.942917,-0.872960,-1.159162,-1.813585,-0.485874,...,-1.387106,-1.699564,-1.454329,-1.394176,-1.810723,-1.293724,-1.472513,-0.856062,-1.417383,-1.518616
2010-01-05,-1.543931,-1.450635,-1.052644,-1.100661,-1.141903,-0.935976,-0.869442,-1.161491,-1.818295,-0.503836,...,-1.389728,-1.682964,-1.456530,-1.393922,-1.736702,-1.302234,-1.474627,-0.856405,-1.363104,-1.490677
2010-01-06,-1.525265,-1.450500,-1.049479,-1.096770,-1.143838,-0.939242,-0.869903,-1.162504,-1.811426,-0.521797,...,-1.400837,-1.639315,-1.440757,-1.391576,-1.753637,-1.300873,-1.479034,-0.857918,-1.363676,-1.419341
2010-01-07,-1.524308,-1.449623,-1.044731,-1.097116,-1.152544,-0.952714,-0.874114,-1.162706,-1.811622,-0.520514,...,-1.373992,-1.649152,-1.443691,-1.385742,-1.726427,-1.303936,-1.479210,-0.862801,-1.323110,-1.319471
2010-01-08,-1.514894,-1.445237,-1.041777,-1.098586,-1.155446,-0.968635,-0.875267,-1.163314,-1.808483,-0.471762,...,-1.346839,-1.638086,-1.444792,-1.378385,-1.735750,-1.303596,-1.479034,-0.863420,-1.361105,-1.335522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-06,-0.292414,1.546442,1.415306,1.519856,1.389115,2.442366,0.914731,1.965188,0.275499,2.139785,...,0.389300,1.453881,1.922642,3.608507,0.319126,1.445091,2.004243,1.049716,0.892462,1.188313
2023-03-07,-0.358751,1.521329,1.333410,1.459713,1.375659,2.346557,0.901926,1.898292,0.244792,2.053947,...,0.333023,1.405703,1.900392,3.568509,0.254160,1.368578,1.928863,1.024616,0.844989,1.021814
2023-03-08,-0.357115,1.551859,1.316262,1.453732,1.383660,2.356884,0.907924,1.909179,0.213232,2.090734,...,0.377114,1.405703,1.923918,3.614530,0.265181,1.410553,1.924429,1.058380,0.804945,1.039038


In [10]:
# Estimate the variance of the standardized prices and compute the eigenvalues and eigenvectors
cov = closeNorm.cov()
eig_vals, eig_vecs = np.linalg.eig(cov)
eig_vals = np.real(eig_vals)
eig_vecs = np.real(eig_vecs)
sorted_idx = np.argsort(eig_vals)

In [11]:
# Plot the cumulative explain variance against increasing PCS
tot = sum(eig_vals)
var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=[df.columns[sorted_idx[i]] for i in range(1, 100)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=[df.columns[sorted_idx[i]] for i in range(1, 100)], 
        y=cum_var_exp,
        name='Cumulative explained variance')

data = (trace1, trace2)

layout = Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance per principal component')

fig = Figure(data=data, layout=layout)

iplot(fig)

Task2: Repeat Task 2 for daily stock returns rather than stock prices. Compare the eigen portfolios of prices and returns. 

In [12]:
# Calculate the linear return from the price of the assets
retDf = df.copy()
retDf.iloc[0, :] = np.zeros(len(retDf.iloc[0, :]))
for i in range(len(retDf) - 1):
    retDf.iloc[i + 1, :] = df.iloc[i, :] / df.iloc[i + 1, :] - 1
retDf

,MMM,AOS,ABT,ACN,ATVI,ADM,ADBE,ADP,AAP,AES,...,WHR,WMB,WTW,GWW,WYNN,XEL,YUM,ZBRA,ZBH,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2010-01-05,0.006303,0.012943,0.008145,-0.006142,-0.001767,-0.005373,-0.016180,0.005399,0.005979,0.010348,...,0.002079,-0.012340,0.002247,-0.000411,-0.057332,0.012002,0.003431,0.001747,-0.030685,-0.034058
2010-01-06,-0.013984,-0.000454,-0.005522,-0.010519,0.005329,0.002535,0.002127,0.002353,-0.008644,0.010455,...,0.008882,-0.031430,-0.015844,-0.003789,0.013291,-0.001917,0.007201,0.007747,0.000323,-0.080000
2010-01-07,-0.000717,-0.002942,-0.008217,0.000936,0.024568,0.010567,0.019789,0.000470,0.000247,-0.000746,...,-0.021015,0.007134,0.002956,-0.009334,-0.020909,0.004331,0.000288,0.025641,-0.022426,-0.100719
2010-01-08,-0.006997,-0.014496,-0.005086,0.003994,0.008257,0.012646,0.005451,0.001415,-0.003937,-0.027576,...,-0.020813,-0.007961,0.001110,-0.011631,0.007216,-0.000481,-0.000288,0.003261,0.021455,0.016454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-06,0.012099,0.018386,0.016545,-0.001595,-0.003014,-0.004177,-0.008587,-0.003414,0.024791,0.004000,...,0.010930,0.008795,0.007698,-0.002944,0.003726,0.000769,0.002862,0.011903,0.012839,0.009639
2023-03-07,0.026520,0.007682,0.027706,0.020553,0.004668,0.020946,0.006439,0.019161,0.010771,0.019992,...,0.018857,0.010866,0.005131,0.007169,0.019795,0.022638,0.018752,0.012080,0.011937,0.049957
2023-03-08,-0.000653,-0.009252,0.005835,0.002048,-0.002768,-0.002253,-0.003007,-0.003109,0.011194,-0.008495,...,-0.014559,0.000000,-0.005396,-0.008181,-0.003347,-0.012267,0.001104,-0.015990,0.010171,-0.005141


In [13]:
# Standardize the close prices
retDf = (retDf - retDf.mean()) / retDf.std()

# Estimate the variance of the standardized prices and compute the eigenvalues and eigenvectors
cov = retDf.cov()
eig_vals, eig_vecs = np.linalg.eig(cov)
eig_vals = np.real(eig_vals)
eig_vecs = np.real(eig_vecs)
sorted_idx = np.argsort(eig_vals)

In [14]:
# Plot the cumulative explain variance against increasing PCS
tot = sum(eig_vals)
var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=[df.columns[sorted_idx[i]] for i in range(1, 100)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=[df.columns[sorted_idx[i]] for i in range(1, 100)], 
        y=cum_var_exp,
        name='Cumulative explained variance')

data = (trace1, trace2)

layout = Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance per principal component')

fig = Figure(data=data, layout=layout)

iplot(fig)

Task 3: To study the stability of the two-year eigen portfolios, conduct a rolling two-year PCA analysis from 2010 until the present. Do so for both daily stock prices and daily stock returns. For analysis consistency, use a one-year overlap between successive periods. Plot the maximum eigenvalues of the two-year covariance matrices year to year. Comment on the stability of the maximum eigenvalues. Comment on the stability of the number of components needed to explain 95% of asset variances. 

In [15]:
# Generate the list of years from 2010-2022
yearList = [x for x in range(2010, 2023)]
yearList

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [16]:
# Iterate through all the years in yearlist and generate the result
closeMaxList = []
returnMaxList = []

for year in yearList:
    tmpPriceDf = df[(df.index.year == year) | (df.index.year == year + 1)]
    
    # Calculate the linear return from the price of the assets for the temporary time stamp
    tmpRetDf = tmpPriceDf.copy()
    tmpRetDf.iloc[0, :] = np.zeros(len(tmpRetDf.iloc[0, :]))
    for i in range(len(tmpRetDf) - 1):
        tmpRetDf.iloc[i + 1, :] = tmpPriceDf.iloc[i, :] / tmpPriceDf.iloc[i + 1, :] - 1
    
    # Normalize the tmpPriceDf and tmpRetDf
    tmpPriceDf = (tmpPriceDf - tmpPriceDf.mean()) / tmpPriceDf.std()
    tmpRetDf = (tmpRetDf - tmpRetDf.mean()) / tmpRetDf.std()

    # Iterate through dataframes and show the result
    dfList = [tmpPriceDf, tmpRetDf]
    print('Result for year ' + str(year) + '-' + str(year + 1))
    tmpNum = 0
    for tmpDf in dfList:
        # Estimate the variance and compute the eigenvalues and eigenvectors
        cov = tmpDf.cov()
        eig_vals, eig_vecs = np.linalg.eig(cov)
        eig_vals = np.real(eig_vals)
        eig_vecs = np.real(eig_vecs)
        sorted_idx = np.argsort(eig_vals)
        if tmpNum == 0:
            closeMaxList.append(df.columns[sorted_idx[0]])
        else:
            returnMaxList.append(df.columns[sorted_idx[0]])
        tmpNum += 1

        # Plot the cumulative explain variance against increasing PCS
        tot = sum(eig_vals)
        var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]
        cum_var_exp = np.cumsum(var_exp)

        trace1 = Bar(
                x=[df.columns[sorted_idx[i]] for i in range(1, 100)],
                y=var_exp,
                showlegend=False)

        trace2 = Scatter(
                x=[df.columns[sorted_idx[i]] for i in range(1, 100)], 
                y=cum_var_exp,
                name='Cumulative explained variance')

        data = (trace1, trace2)

        layout = Layout(
                yaxis=YAxis(title='Explained variance in percent'),
                title='Explained variance per principal component')

        fig = Figure(data=data, layout=layout)

        iplot(fig)

print('STOCK PRICE: ', closeMaxList)
print('STOCK RETURN: ', returnMaxList)

Result for year 2010-2011


Result for year 2011-2012


Result for year 2012-2013


Result for year 2013-2014


Result for year 2014-2015


...

Result for year 2019-2020


Result for year 2020-2021


Result for year 2021-2022


Result for year 2022-2023


CLOSE:  ['RJF ', 'CMCSA ', 'GRMN ', 'RMD ', 'SRE ', 'RTX ', 'SBUX ', 'SPGI ', 'RF ', 'STX ', 'SRE ', 'SPG ', 'NRG ']
MAX:  ['OMC ', 'PAYX ', 'ABC ', 'ILMN ', 'PPG ', 'PXD ', 'POOL ', 'AMT ', 'PNW ', 'AJG ', 'PKG ', 'ATO ', 'NRG ']


Task 4: Repeat Task 3, for weekly stock prices and returns. What are your observations in comparison to the daily eigen portfolios? 

In [46]:
# Define the function to generate the weekly dataframe

def generateWeeklyDataframe(df):
    weekDf = pd.DataFrame(data=None, columns=df.columns)
    tmpList = []
    for i in range(len(df)):
        if len(tmpList) == 0 or df.index[i].date() - df.index[i - 1].date() < datetime.timedelta(2):
            tmpList.append(df.iloc[i])
        else:
            tmp = (sum(tmpList) / len(tmpList)).to_list()
            weekDf.append(tmp)
            tmpList = [df.iloc[i]]
    return weekDf

In [48]:
# Iterate through all the years in yearlist and generate the result
closeMaxList = []
returnMaxList = []

for year in yearList:
    tmpDf = df[(df.index.year == year) | (df.index.year == year + 1)]
    tmpPriceDf = generateWeeklyDataframe(tmpDf)
    print(tmpPriceDf)
    
    # Calculate the linear return from the price of the assets for the temporary time stamp
    tmpRetDf = tmpPriceDf.copy()
    tmpRetDf.iloc[0, :] = np.zeros(len(tmpRetDf.iloc[0, :]))
    for i in range(len(tmpRetDf) - 1):
        tmpRetDf.iloc[i + 1, :] = tmpPriceDf.iloc[i, :] / tmpPriceDf.iloc[i + 1, :] - 1
    
    # Normalize the tmpPriceDf and tmpRetDf
    tmpPriceDf = (tmpPriceDf - tmpPriceDf.mean()) / tmpPriceDf.std()
    tmpRetDf = (tmpRetDf - tmpRetDf.mean()) / tmpRetDf.std()

    # Iterate through dataframes and show the result
    dfList = [tmpPriceDf, tmpRetDf]
    print('Result for year ' + str(year) + '-' + str(year + 1))
    tmpNum = 0
    for tmpDf in dfList:
        # Estimate the variance and compute the eigenvalues and eigenvectors
        cov = tmpDf.cov()
        eig_vals, eig_vecs = np.linalg.eig(cov)
        eig_vals = np.real(eig_vals)
        eig_vecs = np.real(eig_vecs)
        sorted_idx = np.argsort(eig_vals)
        if tmpNum == 0:
            closeMaxList.append(df.columns[sorted_idx[0]])
        else:
            returnMaxList.append(df.columns[sorted_idx[0]])
        tmpNum += 1

        # Plot the cumulative explain variance against increasing PCS
        tot = sum(eig_vals)
        var_exp = [(i / tot) * 100 for i in sorted(eig_vals, reverse=True)]
        cum_var_exp = np.cumsum(var_exp)

        trace1 = Bar(
                x=[df.columns[sorted_idx[i]] for i in range(1, 100)],
                y=var_exp,
                showlegend=False)

        trace2 = Scatter(
                x=[df.columns[sorted_idx[i]] for i in range(1, 100)], 
                y=cum_var_exp,
                name='Cumulative explained variance')

        data = (trace1, trace2)

        layout = Layout(
                yaxis=YAxis(title='Explained variance in percent'),
                title='Explained variance per principal component')

        fig = Figure(data=data, layout=layout)

        iplot(fig)

print('STOCK PRICE: ', closeMaxList)
print('STOCK RETURN: ', returnMaxList)

C:\Users\1\AppData\Local\Temp\ipykernel_8208\3054952243.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppData\Local\Temp\ipykernel_8208\3054952243.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppData\Local\Temp\ipykernel_8208\3054952243.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppData\Local\Temp\ipykernel_8208\3054952243.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppData\Local\Temp\ipykernel_8208\3054952243.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1

Empty DataFrame
Columns: [MMM , AOS , ABT , ACN , ATVI , ADM , ADBE , ADP , AAP , AES , AFL , A , APD , AKAM , ALK , ALB , ARE , ALGN , LNT , ALL , GOOGL , GOOG , MO , AMZN , AMD , AEE , AAL , AEP , AXP , AIG , AMT , AWK , AMP , ABC , AME , AMGN , APH , ADI , ANSS , AON , APA , AAPL , AMAT , ACGL , AJG , AIZ , T , ATO , ADSK , AZO , AVB , AVY , BKR , BALL , BAC , BBWI , BAX , BDX , WRB , BBY , BIO , TECH , BIIB , BLK , BK , BA , BKNG , BWA , BXP , BSX , BMY , AVGO , BR , BRO , CHRW , CDNS , CPT , CPB , COF , CAH , KMX , CCL , CAT , CBRE , CE , CNC , CNP , CF , CRL , SCHW , CVX , CMG , CB , CHD , CI , CINF , CTAS , CSCO , C , CLX , ...]
Index: []

[0 rows x 438 columns]


IndexError: single positional indexer is out-of-bounds